# "kaggle - Geospatial Analysis03"

# 3. Exercise: Interactive Maps


소개
당신은 일본의 도시 안전 계획가이며 일본의 어느 지역에서 추가 지진 보강이 필요한지 분석하고 있습니다. 인구 밀도가 높고 지진이 발생하기 쉬운 지역은 어디입니까?

In [20]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

In [21]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

수업 과정

3.1 지진은 판 경계와 일치합니까?
아래 코드 셀을 실행하여 전역 플레이트 경계를 표시하는 DataFrame plate_boundaries를 만듭니다. "좌표" 열은 경계를 따라 (위도, 경도) 위치의 목록입니다.

In [22]:
plate_boundaries = gpd.read_file("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(-5.444200361999947, 133.6808931800001), (-5...."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(-7.760600482999962, 125.47879802900002), (-7..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(19.817899819000047, 120.09999798800004), (19..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(26.175899215000072, 143.20620700100005), (26..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(0.41880004000006466, 132.8273013480001), (0...."



그런 다음 변경 없이 아래 코드 셀을 실행하여 과거 지진 데이터를 DataFrame 지진으로 로드합니다.

In [23]:
# Load the data and print the first 5 rows
earthquakes = pd.read_csv("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1970-01-04 17:00:40.200,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09
1,1970-01-06 05:35:51.800,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09
2,1970-01-08 17:12:39.100,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09
3,1970-01-10 12:07:08.600,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09
4,1970-01-16 08:05:39.000,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN


아래 코드 셀은 지도에서 판 경계를 시각화합니다. 모든 지진 데이터를 사용하여 동일한 지도에 히트맵을 추가하여 지진이 판 경계와 일치하는지 확인합니다.

In [ ]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# 판 경계 시각화하는 부분
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Add a heatmap to the map
HeatMap(data=earthquakes[['Latitude', 'Longitude']], radius=15).add_to(m_1)

m_1

#### 위 heatmap을 통해 지진이 일어나는 곳과 판 경계가 같다는 것을 알 수 있습니다. 

#### 3-2. 일본의 지진 깊이와 판 경계에 대한 근접성 사이에 관계가 있습니까?

최근에 지진의 깊이가 지구의 구조에 대한 중요한 정보를 알려준다는 것을 읽었습니다. 흥미로운 글로벌 패턴이 있는지 확인하고 싶고 일본의 깊이가 어떻게 다른지 이해하고 싶습니다.

In [ ]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
 

# Custom function to assign a color to each circle
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

# Add a map to visualize earthquake depth
for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        radius=2000,
        color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_2)


m_2

# 50보다 싶이가 작으면 forestgreen, 100보다 작으면 darkorange, 100이상이면 darkred

판 경계와의 근접성과 지진 깊이 사이의 관계를 감지할 수 있습니까? 이 패턴이 전 세계적으로 유지됩니까? 일본에서?

일본 북부에서는 판 경계에 가까운 지진이 더 얕아지는 경향이 있는 것으로 나타났습니다(판 경계에서 멀리 떨어진 지진은 더 깊습니다). 이 패턴은 남미 서부 해안과 같은 다른 지역에서도 반복됩니다. 그러나 모든 곳에서 적용되는 것은 아닙니다(예: 중국, 몽골, 러시아).

#### 3.3 인구밀도가 높은 현은?
변경 없이 다음 코드 셀을 실행하여 일본 현의 지리적 경계를 포함하는 GeoDataFrame 현을 생성합니다.

In [37]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.65330, 137.09546 ..."
Akita,"MULTIPOLYGON (((139.55725 39.20330, 139.55765 ..."
Aomori,"MULTIPOLYGON (((141.39860 40.92472, 141.39806 ..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


다음 코드 셀은 일본 각 현에 대한 인구, 면적(제곱 킬로미터) 및 인구 밀도(제곱 킬로미터당)를 포함하는 DataFrame 통계를 생성합니다. 변경 없이 코드 셀을 실행합니다.

In [38]:
# DataFrame containing population of each prefecture
population = pd.read_csv("C:/Users/Kangdaeyong/Desktop/datamining/kaggle_geospatial_analysis/archive/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# Calculate area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# Add density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


다음 코드 셀을 사용하여 choropleth map을 만들어 인구 밀도를 시각화합니다.

In [ ]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)



# Create a choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__,
           data=stats['density'],
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Population density (per square kilometer)'
          ).add_to(m_3)

m_3

도쿄, 가나가와, 오사카는 인구 밀도가 가장 높습니다. 이들은 모두 일본 중부에 위치하고 있으며 도쿄와 가나가와현이 인접해 있습니다.

#### 3.4. 규모가 큰 지진이 발생하기 쉬운 고밀도 현은?
지진 보강의 혜택을 받을 수 있는 한 현을 제안하는 지도를 만듭니다. 지도는 밀도와 지진 규모를 모두 시각화해야 합니다.

In [ ]:
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Create a map
def color_producer(magnitude):
    if magnitude > 6.5:
        return 'red'
    else:
        return 'green'

Choropleth(
    geo_data=prefectures['geometry'].__geo_interface__,
    data=stats['density'],
    key_on="feature.id",
    fill_color='BuPu',
    legend_name='Population density (per square kilometer)').add_to(m_4)

for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)

m_4

이 질문에 대한 명확하고 단일한 대답은 없지만 몇 가지 합리적인 옵션이 있습니다. 도쿄는 단연 가장 인구 밀도가 높은 현이며 많은 지진을 경험했습니다. 오사카는 상대적으로 인구 밀도가 낮지만 도쿄 근처보다 상대적으로 강한 지진을 경험했습니다. 그리고 가나가와(Kanagawa)의 긴 해안(높은 밀도와 강한 지진의 역사적 근접성에 더하여)은 추가된 잠재적 쓰나미 위험에 대해 걱정하게 만들 수 있습니다.